In [1]:
import pandas as pd
import numpy as np
data='../../data/kundaje/bulk/croo/'

In [2]:
days=['D0','D2','D4','D6','D8','D10','D12','D14','iPSC']
peak='/peak/overlap_reproducibility/overlap.optimal_peak.narrowPeak.gz'

In [3]:
dfs={}
for day in days:
    df=pd.read_table(data+day+peak,header=None)
    df=df[[0,1,2]].drop_duplicates()
    df=df[df[0]!='chrY'][df[0]!='chrX']
    df[0]=df[0].str.strip('chr').astype(int)
    df=df.sort_values([0,1])
    df['d']=df[2]-df[1]
    dfs[day]=df

/home/moqri/anaconda3/envs/pyscenic2/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [4]:
lssds=[]
for ch in range(1,23):
    print(ch,end=',')
    df0=dfs['D0']
    df0=df0[df0[0]==ch]
    df0=df0.sort_values([1])    
    df0=df0[[1,2]].values
    lss=[]
    for day in days:
        df=dfs[day]
        df=df[df[0]==ch]
        df=df[[1,2]].values
        ls=[]
        for i in range(len(df0)):
            r0=df0[i]
            dt=r0.mean()
            dfn=df[(df.mean(1)>dt-1000) & (df.mean(1)<dt+1000)]
            l=0
            for r in dfn:
                com=len(set(range(r0[0],r0[1])).intersection(set(range(r[0],r[1]))))
                l=max(l,com)    
            ls.append(l)    
        lss.append(ls)
    lssd=pd.DataFrame(lss).T
    lssds.append(lssd)
df0=dfs['D0'].copy
df0.columns=['c','b','e','d']
loss=pd.concat([df0.reset_index(drop=True),pd.concat(lssds).reset_index(drop=True)],1)
loss.to_csv('bulk_loss.csv')    

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,

AttributeError: 'method' object has no attribute 'columns'

In [65]:
lssds=[]
for ch in range(1,23):
    print(ch,end=',')
    df0=dfs['iPSC']
    df0=df0[df0[0]==ch]
    df0=df0[[1,2]].values
    lss=[]
    for day in reversed(days):
        df=dfs[day]
        df=df[df[0]==ch]
        df=df[[1,2]].values
        ls=[]
        for i in range(len(df0)):
            r0=df0[i]
            dt=r0.mean()
            dfn=df[(df.mean(1)>dt-1000) & (df.mean(1)<dt+1000)]
            l=0
            for r in dfn:
                com=len(set(range(r0[0],r0[1])).intersection(set(range(r[0],r[1]))))
                l=max(l,com)    
            ls.append(l)    
        lss.append(ls)
    lssd=pd.DataFrame(lss).T
    lssds.append(lssd)
df0=dfs['iPSC'].copy()
df0.columns=['c','b','e','d']
gain=pd.concat([df0.reset_index(drop=True),pd.concat(lssds).reset_index(drop=True)],1)
gain.to_csv('bulk_gain.csv')    

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,

In [30]:
df=pd.read_csv('bulk_loss.csv',index_col=0)
df=pd.read_csv('bulk_gain.csv',index_col=0)

In [59]:
gi=gain[gain['8']>0][['c','b','e']]
gi.shape

(66414, 3)

In [60]:
gi['c']='chr'+gi['c'].astype(str)

In [61]:
gi.to_csv('b0.bed',header=False,sep='\t',index=False)